In [2]:
import tensorflow as tf

from tensorflow.keras.layers import concatenate, BatchNormalization, LSTM 
from tensorflow.keras.layers import Activation, Dropout, Dense, Input
from tensorflow.keras.models import Model
from keras.regularizers import l2

Using TensorFlow backend.


In [97]:
def lstm_encoder(input_shape_1, input_shape_2):
    seq_input_eeg = Input(shape=input_shape_1, name="encoder_ip1")
    seq_input_p = Input(shape=input_shape_2, name="encoder_ip2")
    
    encoded_eeg = LSTM(64, name="encoder_lstm_eeg",return_sequences=True)(seq_input_eeg)
    
    encoded_p = LSTM(32, name="encoder_lstm_p", return_sequences=True)(seq_input_p)
    
    x_combined = concatenate([encoded_eeg, encoded_p], axis = -1)
    
    model = Model(inputs =[seq_input_eeg, seq_input_p], outputs = [x_combined], name="encoder")
    
    return model

In [98]:
def lstm_decoder(input_shape):
    
    x_combined = Input(shape=input_shape, name="decoder_ip")
    
    combined_x = LSTM(64, name="combined_lstm", return_sequences=True)(x_combined)
    
    decoded_eeg = LSTM(64, name="decoder_lstm_eeg",return_sequences=True)(combined_x)
    
    decoded_p = LSTM(32, name="decoder_lstm_p", return_sequences=True)(x_combined)
    
    reconstructed_eeg = LSTM(128, name="reconstruct_lstm_eeg", return_sequences=True)(decoded_eeg)
    
    reconstructed_p = LSTM(48, name="reconstruct_lstm_p", return_sequences=True)(decoded_p)
    
    model = Model(inputs = x_combined, outputs=[reconstructed_eeg, reconstructed_p], name="decoder")
    return model

In [99]:
encoder = lstm_encoder((10,128),(10,48))

In [100]:
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_ip1 (InputLayer)        [(None, 10, 128)]    0                                            
__________________________________________________________________________________________________
encoder_ip2 (InputLayer)        [(None, 10, 48)]     0                                            
__________________________________________________________________________________________________
encoder_lstm_eeg (LSTM)         (None, 10, 64)       49408       encoder_ip1[0][0]                
__________________________________________________________________________________________________
encoder_lstm_p (LSTM)           (None, 10, 32)       10368       encoder_ip2[0][0]                
____________________________________________________________________________________________

In [101]:
decoder = lstm_decoder((10,96))

In [102]:
decoder.summary()

Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_ip (InputLayer)         [(None, 10, 96)]     0                                            
__________________________________________________________________________________________________
combined_lstm (LSTM)            (None, 10, 64)       41216       decoder_ip[0][0]                 
__________________________________________________________________________________________________
decoder_lstm_eeg (LSTM)         (None, 10, 64)       33024       combined_lstm[0][0]              
__________________________________________________________________________________________________
decoder_lstm_p (LSTM)           (None, 10, 32)       16512       decoder_ip[0][0]                 
____________________________________________________________________________________________

In [115]:
def lstm_autoencoder(input_shape_1, input_shape_2, encoder, decoder):
    
    ip1 = Input(shape=input_shape_1, name="ip1")
    
    ip2 = Input(shape=input_shape_2, name="ip2")
    
    encoded = encoder([ip1, ip2])
    
    decoded = decoder(encoded)
    
    model = Model(inputs = [ip1, ip2], outputs = decoded, name="autoencoder")
    
    return model

In [116]:
autoencoder = lstm_autoencoder((10,128),(10,48), encoder, decoder)

In [117]:
autoencoder.summary()

Model: "autoencoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ip1 (InputLayer)                [(None, 10, 128)]    0                                            
__________________________________________________________________________________________________
ip2 (InputLayer)                [(None, 10, 48)]     0                                            
__________________________________________________________________________________________________
encoder (Model)                 (None, 10, 96)       59776       ip1[0][0]                        
                                                                 ip2[0][0]                        
__________________________________________________________________________________________________
decoder (Model)                 [(None, 10, 128), (N 205120      encoder[3][0]          